In [1]:
!pip install pyocr
!pip install pytesseract
!apt install tesseract-ocr
!apt install -y tesseract-ocr-jpn
!apt install -y tesseract-ocr-jpn-vert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 1.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,657 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123621 file

In [ ]:
import pyocr
import pytesseract
from PIL import Image

tools = pyocr.get_available_tools()
tool = tools[0]

img = Image.open('path_to_image')  # 実際のファイルパスに変更してください

text = tool.image_to_string(
    img,
    lang="jpn",
    builder=pyocr.builders.TextBuilder(tesseract_layout=6)
)

print(text)

In [ ]:
import re
import openpyxl
import os
from datetime import datetime

# 数値や日付の正規表現パターン
amountPattern = r'\\\\?(\\d+\\.\\d{3})'
dateOfUsePattern = r'\\d{4}/\\d{2}/\\d{2}'

# ファイルパスをプレースホルダー化
file_path = 'path_to_excel_file.xlsx'
if os.path.exists(file_path):
    existing_workbook = openpyxl.load_workbook(file_path)
else:
    existing_workbook = openpyxl.Workbook()

# シートを取得
sheet = existing_workbook.active

# 一番番号が大きいセルを探す
max_row = sheet.max_row if sheet.max_row else 0

# マッチするすべてのパターンを抽出
amountMatches = re.findall(amountPattern, text)
dateOfUseMatch = re.search(dateOfUsePattern, text)

# 正しいデータがある場合にExcelに書き込み
if dateOfUseMatch and len(set(amountMatches)) == 1:
    date_of_use = dateOfUseMatch.group(0)
    sheet.cell(row=13, column=1, value=date_of_use)
    print(f"Date of use {date_of_use} added to cell A13")

    result = amountMatches[0].replace('.', '')
    result_numeric = float(result)
    sheet.cell(row=13, column=4, value=result_numeric)
    print(f"Value {result} added to cell A{max_row + 1}")

    today_date = datetime.now().strftime('%Y/%m/%d')
    sheet['F3'] = today_date
    print(f"Today's date {today_date} added to cell F3")

    # 年と月を抽出し、新しいファイル名に設定
    year, month = re.match(r'(\\d{4})/(\\d{2})/\\d{2}', date_of_use).groups()
    new_file_name = f'/content/drive/MyDrive/{year}年{int(month)}月分_立替経費精算.xlsx'

    # Excelファイルを新しい名前で保存
    existing_workbook.save(new_file_name)
    print(f"File saved as {new_file_name}")
else:
    print("No date or amount found.")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
